Fig. 5: Benchmark FateID
----

In this notebook, we extract FateID's fate biases and smoothed expression.

# Preliminaries

## Import packages

In [1]:
# import standard packages
import numpy as np
import pandas as pd
from rpy2.robjects import ListVector

import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

# import single-cell packages
import scanpy as sc
import scanpy.external as sce
import scvelo as scv
import cellrank as cr
import anndata2ri

# set verbosity levels
sc.settings.verbosity = 2
cr.settings.verbosity = 2
scv.settings.verbosity = 3 

anndata2ri.activate()
%load_ext rpy2.ipython

## Print package versions for reproducibility

If you want to exactly reproduce the results shown here, please make sure that your package versions match what is printed below. 

In [2]:
cr.logging.print_versions()

cellrank==1.0.0-rc.12 scanpy==1.6.0 anndata==0.7.4 numpy==1.19.2 numba==0.51.2 scipy==1.5.2 pandas==1.1.3 scikit-learn==0.23.2 statsmodels==0.12.0 python-igraph==0.8.3 scvelo==0.2.2 pygam==0.8.0 matplotlib==3.3.2 seaborn==0.11.0


## Set up paths

Define the paths to load data, cache results and write figure panels.

In [13]:
sys.path.insert(0, "../../../")  # this depends on the notebook depth and must be adapted per notebook

from paths import DATA_DIR, CACHE_DIR, FIG_DIR

## Set global parameters

In [1]:
root = str(DATA_DIR / "benchmarking" / "fateid")

NameError: name 'DATA_DIR' is not defined

If there are other global parameters for this analysis, put them here as well. 

## Define utility functions

In [9]:
%%R
# copy of plotexpressionProfile which extracts the LOESSed expression
extractData <- function (x, y, g, n, logsc = FALSE, col = NULL, name = NULL, cluster = FALSE, alpha = 0.5, lwd = 1, ylim = NULL){
    
    if ( logsc ){
        if ( min(x) == 0 ) x <- x + .1
        x <- x/apply(x,1,sum)
        x <- log2(x)
    }else{
        x <- x/apply(x,1,sum)
    }
    
    cl <- unique(y[n])
    set.seed(111111)
    if (is.null(col)) col <- sample(rainbow(length(n)))
    xlim <- c(1, length(n))
    z  <- x[g, n]
    zc <- z
    u <- 1:length(n)
       
    for ( i in 1:nrow(z) ){
        v <- as.vector(t(z[i,]))
        k <- predict(loess(v ~ u, span = alpha))
        if ( ! logsc ) k[k < 0] <- 0
        zc[i,] <- k
    }
    
    return(zc)
}

dptTraj <- function(x,y,fb,root_idxs,trthr=NULL,distance="euclidean",sigma=1000,...){
  trc <- list()
  pts <- list()  
  for ( j in colnames(fb$probs) ){
    if ( ! is.null(trthr) ){
      probs <- fb$probs
      n  <- rownames(probs)[probs[,j] > trthr]
    }else{
      votes <- fb$votes
      b <- bias(votes)
      n  <- rownames(votes)[b$bias[,j] > 1 & b$pv < .05]
    }
    dm <- destiny::DiffusionMap(as.matrix(t(x[,n])),distance=distance,sigma=sigma,...)
    root_idx <- root_idxs[[j]]
    pt <- destiny::DPT(dm, root_idx)
    pto <- pt[root_idx, ]
    
    #b <- pt@branch[, 1]
    #tip_idx <- which(b==1 & !is.na(b) & pt@tips[, 1])
    #pto <- pt[tip_idx, ]
    
    od <- order(pto,decreasing=FALSE)
    n <- n[od]

    #ts <- Transitions(as.matrix(t(x[,n])),distance=distance,sigma=sigma,...)
    #pt <- dpt::dpt(ts, branching = FALSE)
    #n <- n[order(pt$DPT,decreasing=FALSE)]
  
    if ( median((1:length(n))[y[n] == sub("t","",j)]) < median((1:length(n))[y[n] != sub("t","",j)]) ) {
        n <- rev(n)
        od <- rev(od)
    }
    trc[[j]] <- n
    pts[[j]] <- pto[od]
  }
  return(list(trc=trc,pts=pts))
}

# reason:
# Error in bias(votes) : could not find function "bias"
bias <- function(tvn){
  bias <- tvn/apply(tvn,1,function(x) x[order(x,decreasing=TRUE)][2] + 1e-3)
  pv <- apply(tvn,1,function(x){ h <- x[order(x,decreasing=TRUE)][1];  l <- x[order(x,decreasing=TRUE)][2]; binom.test( c(h, l), alternative="t" )$p.value})
  return(list(bias=bias,pv=pv))
}

In [10]:
def run_dpt():
    adata_preprocessed = cr.datasets.pancreas(DATA_DIR / "pancreas" / "pancreas_preprocessed.h5ad")
    # TODO: this can be removed (already have a DPT in the object)
    # del adata_preprocessed.uns["neighbors"]
    # adata_preprocessed.uns['iroot'] = np.where(adata_preprocessed.obs_names == 'TGCTACCCAGGGCATA-1-3')[0][0]
    # sc.tl.diffmap(adata_preprocessed)
    # sc.tl.dpt(adata_preprocessed)
    # sc.pl.umap(adata_preprocessed, color='dpt_pseudotime')
    
    return adata_preprocessed

## Load the data

Load the AnnData object from the CellRank software package. 

In [19]:
adata = cr.datasets.pancreas(DATA_DIR / "pancreas" / "pancreas.h5ad")
del adata.uns["neighbors"]  # creshed anndata2ri, we don't need it
adata

AnnData object with n_obs × n_vars = 2531 × 27998
    obs: 'day', 'proliferation', 'G2M_score', 'S_score', 'phase', 'clusters_coarse', 'clusters', 'clusters_fine', 'louvain_Alpha', 'louvain_Beta'
    var: 'highly_variable_genes'
    uns: 'clusters_colors', 'clusters_fine_colors', 'day_colors', 'louvain_Alpha_colors', 'louvain_Beta_colors', 'pca'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

# Analysis

## Convert AnnData to SCseq

In [26]:
%%R -i adata -i root
out_dir = root
X <- adata@assays@data[['X']]  # X, spliced, unspliced
rownames(X) <- rownames(adata)
colnames(X) <- colnames(adata)
# The raw expression data matrix with cells as columns and genes as rows in sparse matrix format.
sc <- SCseq(X)

R[write to console]: Error in SCseq(X) : could not find function "SCseq"




Error in SCseq(X) : could not find function "SCseq"


## Filter, cluster and compute the embedding

In [ ]:
%%R
sc <- filterdata(sc, mintotal=10)
sc <- compdist(sc, metric="logpearson")
sc <- clustexp(sc, samp=1000)
sc <- findoutliers(sc)
sc <- comptsne(sc)

## Extract data needed for fateBias

In [ ]:
%%R
x <- as.matrix(getfdata(sc)[sc@cluster$features,])
print(dim(x))
genes <- c("Pax4", "Pdx1", "Arx", "Peg10", "Irs4" ,"Ghrl", "Hhex", "Cd24a")
for (gene in genes) {
    if (!gene %in% rownames(x)) {
        print(gene)
        rnames <- rownames(x)
        x <- rbind(x, getfdata(sc)[gene,])
        rownames(x) <- c(rnames, gene)
    }
}
for (gene in genes) {
    if (!gene %in% rownames(x)) {
        print(gene)
    }
}
print(dim(x))

markers <- list(Beta=c("Ins1"), Alpha=c("Gcg"), Epsilon=c("Ghrl"), Delta=c("Sst"))

# cell type partition based, n - positive integer number. For each component of FMarker the expression
# of all genes is aggregated in every cell and the n top-expressing cells are extracted.
pa <- getPart(x, markers, n=50)

# A vector with a partitioning, i. e. cluster assignment for each cell
clustering <- pa$part
# A vector with the numbers of target (end point) clusters.
# Cluster 1 comprises all cells with no enrichment of marker genes
endpoints <- pa$tar
z <- sc@distances

## Run fateBias

In [ ]:
%%R
# fateBias(x, y, tar, z=NULL, minnr=5, minnrh=10, adapt=TRUE, confidence=0.75, nbfactor=5, use.dist=FALSE, seed=12345, nbtree=NULL)
# minr - step size of the algo (test size - number of cells to be classified per end point, recommended 5)
# minnrh - cells used for training (can be inf to use all previous cells, recommended: 20)
# x - data representation (e.g. could be only HVGs, PCA, ...)

# as.matrix(x) necessary if use.dist=FALSE
fb  <- fateBias(as.matrix(x), clustering, endpoints, z=z, minnr=5, minnrh=20, seed=123, use.dist=FALSE)

### Extract the values

In [ ]:
%%R -o res
trthr = 0.15  # fate bias threshold
res = list()
for ( j in colnames(fb$probs) ){
  probs <- fb$probs
  n  <- rownames(probs)[probs[,j] > trthr]
  res[[j]] = n
}

### Identify root cells witin each lineage

In [ ]:
adata_preprocessed = run_dpt()
root_names = []

for i in range(2, 6):
    indices = res.rx2(f"t{i}")
    tmp = adata_preprocessed[indices].obs['dpt_pseudotime']
    ix = tmp.argmin()
    root_names.append(tmp.index[ix])
    
print(root_names)

root_ixs = []
for root_name, key in zip(root_names, [f"t{i}" for i in range(2, 6)]):
    # +1 because R indexes from 1
    root_ixs.append(np.where(res.rx2(key) == root_name)[0][0] + 1)
root_ixs = dict(zip([f"t{i}" for i in range(2, 6)], root_ixs))
print(root_ixs)
root_ixs = ListVector(root_ixs)

## Run DPT

In [ ]:
%%R -i root_ixs
trc <- dptTraj(as.matrix(x), clustering, fb, root_ixs, trthr=trthr, distance="euclidean")
pts <- trc[['pts']]
trc <- trc[['trc']]
names(trc) <- c('Beta', 'Alpha', 'Epsilon', 'Delta')
names(pts) <- c('Beta', 'Alpha', 'Epsilon', 'Delta')

## Extract and save the data for plotting

### Save the smoothed expression values and DPT

In [ ]:
%%R
for (lineage in names(trc)) {
    n <- trc[[lineage]]
    fs <- filterset(x,n=n, minexpr=0)
    for (gene in genes) {
        stopifnot(gene %in% rownames(fs))
    }
    # this is the same as plotexpressionProfile, but we return the LOESS'd data
    tmp <- extractData(as.matrix(x), clustering, g=genes, n=n, cluster=FALSE, logsc=FALSE)
    
    write.csv(t(tmp), paste0(out_dir, "/", lineage, ".csv"))
    write.csv(pts[[lineage]], paste0(out_dir, "/", lineage, "_pt.csv"))
    write.csv(trc[[lineage]], paste0(out_dir, "/", lineage, "_ixs.csv"))
}

### Save fate biases and TSNE

In [ ]:
%%R
# taken from Palantir's notebooks
write.csv(sc@tsne, paste0(out_dir, "/", "tsne.csv"))
write.csv(fb$probs, paste0(out_dir, "/", "probs.csv"))
write.csv(sc@cluster$kpart, paste0(out_dir, "/", "clusters.csv"))
for (i in 1:length(markers)){
    order = fb$tr[sprintf("t%d", i+1)]
    write.csv(order, sprintf("%s/%s_order.csv", out_dir, names(markers)[i]))
}

## Visualize smoothed trends for the Delta lineage

In [15]:
%%R
print(lineage)
for (gene in genes) {
    stopifnot(gene %in% rownames(x))
}
plotexpressionProfile(x, clustering, n=n, g=genes, cluster=FALSE, logsc=FALSE)

R[write to console]: Error in print(lineage) : object 'lineage' not found

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/local/lib/R/site-library’ contains no packages




Error in print(lineage) : object 'lineage' not found
